In [1]:
import sqlite3 as sql
import pandas as pd

con = sql.connect("booking.sqlite")
b_damp = open("booking.db", "r", encoding='utf-8-sig')
damp = b_damp.read()
b_damp.close()
con.executescript(damp)

In [37]:
df = pd.read_sql('''
SELECT room_name,
       check_in_date,
       check_out_date,
       JULIANDAY(check_out_date) - JULIANDAY(check_in_date) + 1 as Дни,
       price * (JULIANDAY(check_out_date) - JULIANDAY(check_in_date) + 1) as Счет
FROM room_booking
     JOIN room USING (room_id)
     JOIN type_room USING (type_room_id) 
     JOIN status USING (status_id)
WHERE type_room_name = :s_type 
     AND status_name = :s_status
ORDER BY room_name, price DESC, check_out_date DESC
''', con, params={"s_type": "Стандартный двухместный номер", "s_status": "Занят"})
print(df)

   room_name check_in_date check_out_date   Дни     Счет
0     С-0201    2020-10-30     2020-11-06   8.0  20000.0
1     С-0203    2020-11-07     2020-11-10   4.0  10000.0
2     С-0205    2021-02-01     2021-02-11  11.0  27500.0
3     С-0205    2020-12-02     2020-12-03   2.0   5000.0
4     С-0207    2021-01-20     2021-01-23   4.0  10000.0
5     С-0209    2021-02-11     2021-02-13   3.0   7500.0
6     С-0213    2021-03-07     2021-03-12   6.0  15000.0
7     С-0213    2021-02-07     2021-02-17  11.0  27500.0
8     С-0213    2020-11-25     2020-12-03   9.0  22500.0
9     С-0213    2020-09-26     2020-09-29   4.0  10000.0
10    С-0215    2021-03-12     2021-03-18   7.0  17500.0
11    С-0215    2021-01-26     2021-01-28   3.0   7500.0
12    С-0215    2020-11-20     2020-11-23   4.0  10000.0
13    С-0217    2021-01-09     2021-01-15   7.0  17500.0
14    С-0217    2020-12-28     2021-01-07  11.0  27500.0
15    С-0217    2020-12-11     2020-12-24  14.0  35000.0
16    С-0217    2020-09-28     

In [2]:
df = pd.read_sql('''
SELECT guest_name as Гость,
       GROUP_CONCAT (service_name, CHAR(10)) as Услуги,
       SUM(price) as Сумма
FROM (SELECT *
         FROM service_booking
         JOIN room_booking USING (room_booking_id)
         JOIN guest USING (guest_id)
         JOIN service USING (service_id)
         ORDER BY service_name)       
WHERE service_start_date BETWEEN :s_left_date AND :s_right_date
GROUP BY guest_name
''', con, params={"s_left_date": "2020-12-01", "s_right_date":"2021-01-31"})
print(df)

            Гость                                             Услуги  Сумма
0   Абрамова А.А.                                               Сейф    100
1    Астахов И.И.  Бизнес-центр\nСпа и оздоровительный центр\nСпа...  22431
2    Жидкова Р.Л.                           Трансфер от/до аэропорта   1882
3     Садиев С.И.                Трансфер от/до аэропорта\nЭкскурсии   5000
4    Самарин С.С.                                       Фитнес-центр   1459
5    Солиева К.П.                    Доставка еды и напитков в номер   4815
6  Тихомиров Т.М.                                   Услуги прачечной    100
7    Тощаков П.С.                    Доставка еды и напитков в номер   3259
8    Туполев И.Д.                                   Услуги прачечной    383
9    Шевцова А.Р.                  Сейф\nСпа и оздоровительный центр   3719


In [2]:
df = pd.read_sql('''
WITH get_guest_count(guest_id, c_guest, count) AS(
    SELECT guest_id, guest_name, COUNT(guest_name)
    FROM guest 
    JOIN room_booking USING (guest_id)
    GROUP BY guest_name
),
    get_max_count(max_count) AS (
    SELECT MAX(count)
    FROM get_guest_count
),  get_guest_with_max_count(guest_id, m_guest) AS (
    SELECT get_guest_count.guest_id, get_guest_count.c_guest 
    FROM get_guest_count, get_max_count
    WHERE get_max_count.max_count = get_guest_count.count
)
SELECT m_guest as Гость,
        room_name as Номер,
        COUNT(room_name) as Количество,
        type_room_name as Тип_номера
FROM get_guest_with_max_count 
    JOIN room_booking USING (guest_id)
    JOIN room USING (room_id)
    JOIN type_room USING (type_room_id)
GROUP BY room_name
ORDER BY m_guest
''', con)

print(df)

           Гость   Номер  Количество                           Тип_номера
0   Астахов И.И.  П-0815           1  Представительский двухместный номер
1   Астахов И.И.  П-1004           2  Представительский одноместный номер
2   Астахов И.И.  С-0204           1      Двухместный номер бизнес класса
3   Астахов И.И.  С-0206           1      Двухместный номер бизнес класса
4   Астахов И.И.  С-0215           1        Стандартный двухместный номер
5   Астахов И.И.  С-0217           1        Стандартный двухместный номер
6   Астахов И.И.  С-0219           1        Стандартный одноместный номер
7   Астахов И.И.  С-0220           1      Одноместный номер бизнес класса
8   Астахов И.И.  С-0222           2      Одноместный номер бизнес класса
9   Астахов И.И.  С-0225           1        Стандартный одноместный номер
10  Астахов И.И.  С-0226           1      Одноместный номер бизнес класса


In [6]:
# CREATE TABLE IF NOT EXISTS bill
# AS 
# WITH get_guest_info(guest_info, deposit) AS(
#     SELECT guest_name || " " || room_name || " " || check_in_date || "/" || check_out_date, 15000
#     FROM guest
#      JOIN room_booking USING (guest_id)
#      JOIN room USING (room_id)
# WHERE guest_name = "Астахов И.И." AND room_name = "С-0206" AND check_in_date = "2021-01-13"
# ), get_services(services, sum) AS (
# SELECT service_name || " " || GROUP_CONCAT(service_start_date), SUM(price)
# FROM service
#      JOIN service_booking USING(service_id)
#      JOIN room_booking USING(room_booking_id)
#      JOIN room USING(room_id)
#      JOIN guest USING(guest_id)
# WHERE guest_name = "Астахов И.И." AND room_name = "С-0206" AND check_in_date = "2021-01-13"
# GROUP BY service_name
# ), get_total(info, value) AS(
#     SELECT CASE
#         WHEN get_guest_info.deposit - SUM(get_services.sum) > 0 THEN "Вернуть"
#         WHEN get_guest_info.deposit - SUM(get_services.sum) < 0 THEN "Доплатить" 
#         ELSE "Итого" 
#             END,
#         ABS(get_guest_info.deposit - SUM(get_services.sum))
#     FROM get_services, get_guest_info
# ), get_report(report, sum) AS(
#     SELECT *
#     FROM get_guest_info
#     UNION ALL
#     SELECT *
#     FROM get_services
#     UNION ALL
#     SELECT *
#     FROM get_total)
# SELECT report as Вид_платежа, sum as Сумма FROM get_report;

df = pd.read_sql("""
SELECT * FROM bill
""", con)           
print(df)

                                 Вид_платежа  Сумма
0  Астахов И.И. С-0206 2021-01-13/2021-01-16  15000
1     Спа и оздоровительный центр 2021-01-15   2755
2        Трансфер от/до аэропорта 2021-01-16    900
3            Экскурсии 2021-01-14,2021-01-15  12700
4                                  Доплатить   1355


In [55]:
df = pd.read_sql("""
SELECT strftime('%m', service_start_date) as Месяц,
       service_start_date as Дата,
       service_name as Услуга,
       price as Сумма,
       SUM(price) OVER p_price AS Сумма_с_накоплением
FROM service_booking
    JOIN service USING(service_id)
WHERE strftime('%Y', service_start_date) = "2020"
WINDOW p_price AS(
    PARTITION BY strftime('%m', service_start_date)
    ORDER BY service_start_date, service_name
    ROWS UNBOUNDED PRECEDING
)

""", con)           
print(df)

   Месяц        Дата                           Услуга  Сумма  \
0     09  2020-09-27  Доставка еды и напитков в номер   8665   
1     09  2020-09-28         Трансфер от/до аэропорта   1050   
2     09  2020-09-28                        Экскурсии   4089   
3     09  2020-09-29                     Бизнес-центр   1000   
4     10  2020-10-04  Доставка еды и напитков в номер   2343   
5     10  2020-10-04      Спа и оздоровительный центр   4448   
6     10  2020-10-05         Трансфер от/до аэропорта   1106   
7     10  2020-10-17                     Бизнес-центр   1000   
8     10  2020-10-30         Трансфер от/до аэропорта   1000   
9     11  2020-11-06         Трансфер от/до аэропорта   1200   
10    11  2020-11-08                     Бизнес-центр   1000   
11    11  2020-11-09                     Бизнес-центр   1000   
12    11  2020-11-10                        Экскурсии   3980   
13    11  2020-11-16                        Экскурсии   2595   
14    11  2020-11-18                    